# Sample Evaluations

This notebook demonstrates how to use methods to create, get, and list cloud evaluations using the Azure AI Project SDK.

## Prerequisites

Before running this notebook, ensure you have the following:
- Python 3.8 or later.
- The Azure AI Project SDK installed. You can install it using the following command:
  ```bash
  pip install azure-ai-projects azure-identity
  ```
- Set the following environment variables with your own values:
  - `PROJECT_ENDPOINT`: The Azure AI Project endpoint, as found in the overview page of your Azure AI Foundry project.
  - `DATASET_NAME`: The name of the dataset to create and use in this sample.

In [ ]:
# Import required libraries
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    Evaluation,
    InputDataset,
    EvaluatorConfiguration,
    EvaluatorIds
)

## Set Environment Variables

Ensure the following environment variables are set before proceeding.

In [ ]:
# Set environment variables
endpoint = os.environ["PROJECT_ENDPOINT"]  # Example: https://<account_name>.services.ai.azure.com/api/projects/<project_name>
model_endpoint = os.environ["MODEL_ENDPOINT"]  # Example: https://<account_name>.services.ai.azure.com
model_api_key = os.environ["MODEL_API_KEY"]
model_deployment_name = os.environ["MODEL_DEPLOYMENT_NAME"]  # Example: gpt-4o-mini
dataset_name = os.environ["DATASET_NAME"]

## Authenticate and Initialize the Client

In [ ]:
# Authenticate and initialize the client
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
project_client = AIProjectClient(endpoint=endpoint, credential=credential)

## Upload a Dataset

Upload a single file and create a new dataset to reference the file. Here, we explicitly specify the dataset version.

In [ ]:
# Upload a dataset
print("Uploading dataset...")
dataset = project_client.datasets.upload_file(
    name=dataset_name,
    version="1",
    file="./samples_folder/sample_data_evaluation.jsonl",
)
print("Dataset uploaded:", dataset)

## Create an Evaluation

Define common data mapping and initialization parameters that will apply to all evaluators.

In [ ]:
# Define common data mapping and initialization parameters
common_data_mapping = {
    "query": "${data.query}",
    "response": "${data.response}",
}

common_init_params = {
    "deployment_name": model_deployment_name,
}

# Create an evaluation
evaluation = Evaluation(
    display_name="Sample Evaluation Test",
    description="Sample evaluation for testing",
    data=InputDataset(id="<>"),
    data_mapping=common_data_mapping,
    init_params=common_init_params,
    evaluators={
        "relevance": EvaluatorConfiguration(
            id=EvaluatorIds.RELEVANCE.value,
        ),
        "violence": EvaluatorConfiguration(
            id=EvaluatorIds.VIOLENCE.value,
        ),
        "bleu_score": EvaluatorConfiguration(
            id=EvaluatorIds.BLEU_SCORE.value,
        ),
    },
)

evaluation_response = project_client.evaluations.create(
    evaluation,
    headers={
        "model-endpoint": model_endpoint,
        "api-key": model_api_key,
    },
)
print("Evaluation created:", evaluation_response)

## Get an Evaluation

Retrieve the details of the created evaluation.

In [ ]:
# Get evaluation details
print("Getting evaluation details...")
get_evaluation_response = project_client.evaluations.get(evaluation_response.name)
print("Evaluation details:", get_evaluation_response)

## List Evaluations

List all evaluations in the project.

In [ ]:
# List all evaluations
print("Listing all evaluations...")
for evaluation in project_client.evaluations.list():
    print(evaluation)